In [182]:
import urllib.request

capitals = []
coordinates = {}

'''
c105  = https://people.idsia.ch/~luca/macs-vrptw/problems/c105.txt
c205  = https://people.idsia.ch/~luca/macs-vrptw/problems/c205.txt
r101  = https://people.idsia.ch/~luca/macs-vrptw/problems/r101.txt
r111  = https://people.idsia.ch/~luca/macs-vrptw/problems/r111.txt
r201  = https://people.idsia.ch/~luca/macs-vrptw/problems/r201.txt
rc201 = https://people.idsia.ch/~luca/macs-vrptw/problems/rc201.txt
'''

target_url = "https://people.idsia.ch/~luca/macs-vrptw/problems/c101.txt"

line_number = 0
for line in urllib.request.urlopen(target_url):
    #print(line.decode('utf-8'))
    line_number += 1
    
    if line_number >= 9:
        CUSTOMER = [float(s) for s in line.split()]
        #print("list_: ", list_)
        if len(CUSTOMER) > 0:
            who = str(CUSTOMER[0])
            capitals.append(who)
            coordinates[who] = (CUSTOMER[1], CUSTOMER[2])

In [183]:
'''
l = [(60.0, 50.0), (20.0, 50.0), (40.0, 75.0), (40.0, 25.0), (20.0, 25.0), (20.0, 75.0), (60.0, 25.0), (60.0, 75.0)]

for el in l:
    capitals.append(str(float(capitals[-1])+1))
    print(el)
    coordinates[capitals[-1]] = el
'''

'\nl = [(60.0, 50.0), (20.0, 50.0), (40.0, 75.0), (40.0, 25.0), (20.0, 25.0), (20.0, 75.0), (60.0, 25.0), (60.0, 75.0)]\n\nfor el in l:\n    capitals.append(str(float(capitals[-1])+1))\n    print(el)\n    coordinates[capitals[-1]] = el\n'

In [184]:
print("capitals: ")
i = 0
for el in capitals:
  print("by i = ", i, " capital: ", el, " it's coordinates: {", coordinates[el][0], coordinates[el][1], "}")
  i += 1

capitals: 
by i =  0  capital:  0.0  it's coordinates: { 40.0 50.0 }
by i =  1  capital:  1.0  it's coordinates: { 45.0 68.0 }
by i =  2  capital:  2.0  it's coordinates: { 45.0 70.0 }
by i =  3  capital:  3.0  it's coordinates: { 42.0 66.0 }
by i =  4  capital:  4.0  it's coordinates: { 42.0 68.0 }
by i =  5  capital:  5.0  it's coordinates: { 42.0 65.0 }
by i =  6  capital:  6.0  it's coordinates: { 40.0 69.0 }
by i =  7  capital:  7.0  it's coordinates: { 40.0 66.0 }
by i =  8  capital:  8.0  it's coordinates: { 38.0 68.0 }
by i =  9  capital:  9.0  it's coordinates: { 38.0 70.0 }
by i =  10  capital:  10.0  it's coordinates: { 35.0 66.0 }
by i =  11  capital:  11.0  it's coordinates: { 35.0 69.0 }
by i =  12  capital:  12.0  it's coordinates: { 25.0 85.0 }
by i =  13  capital:  13.0  it's coordinates: { 22.0 75.0 }
by i =  14  capital:  14.0  it's coordinates: { 22.0 85.0 }
by i =  15  capital:  15.0  it's coordinates: { 20.0 80.0 }
by i =  16  capital:  16.0  it's coordinates: { 2

In [185]:
import math
from itertools import combinations

# Compute pairwise distance matrix

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}

In [186]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7 & Gurobi 9.0.0

m = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Symmetric direction: Copy the object
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

# Constraints: two edges incident to each city
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

In [187]:
# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

In [188]:
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-4460  CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 101 rows, 5050 columns and 10100 nonzeros
Model fingerprint: 0xea011af3
Variable types: 0 continuous, 5050 integer (5050 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 101 rows, 5050 columns, 10100 nonzeros
Variable types: 0 continuous, 5050 integer (5050 binary)

Root relaxation: objective 3.586819e+02, 129 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  358.68187    0   14          -  358.68187  

In [189]:
# Retrieve solution

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capitals)

In [190]:
points = []
for city in tour:
  points.append(coordinates[city])
points.append(points[0])

In [191]:
print("tour: ")
for p in points:
  print("{", p[0], p[1], "}")

tour: 
{ 40.0 50.0 }
{ 30.0 50.0 }
{ 25.0 50.0 }
{ 25.0 52.0 }
{ 23.0 52.0 }
{ 20.0 50.0 }
{ 8.0 45.0 }
{ 5.0 45.0 }
{ 0.0 45.0 }
{ 0.0 40.0 }
{ 2.0 40.0 }
{ 5.0 35.0 }
{ 8.0 40.0 }
{ 10.0 40.0 }
{ 10.0 35.0 }
{ 25.0 35.0 }
{ 26.0 32.0 }
{ 25.0 30.0 }
{ 28.0 30.0 }
{ 30.0 30.0 }
{ 32.0 30.0 }
{ 30.0 32.0 }
{ 28.0 35.0 }
{ 30.0 35.0 }
{ 33.0 35.0 }
{ 33.0 32.0 }
{ 35.0 32.0 }
{ 35.0 30.0 }
{ 38.0 15.0 }
{ 35.0 5.0 }
{ 38.0 5.0 }
{ 40.0 5.0 }
{ 44.0 5.0 }
{ 42.0 10.0 }
{ 40.0 15.0 }
{ 42.0 15.0 }
{ 45.0 30.0 }
{ 48.0 30.0 }
{ 50.0 30.0 }
{ 53.0 30.0 }
{ 50.0 35.0 }
{ 47.0 35.0 }
{ 45.0 35.0 }
{ 47.0 40.0 }
{ 48.0 40.0 }
{ 50.0 40.0 }
{ 53.0 35.0 }
{ 85.0 25.0 }
{ 87.0 30.0 }
{ 88.0 30.0 }
{ 92.0 30.0 }
{ 95.0 30.0 }
{ 95.0 35.0 }
{ 90.0 35.0 }
{ 88.0 35.0 }
{ 85.0 35.0 }
{ 75.0 55.0 }
{ 72.0 55.0 }
{ 70.0 58.0 }
{ 68.0 60.0 }
{ 65.0 60.0 }
{ 63.0 58.0 }
{ 66.0 55.0 }
{ 65.0 55.0 }
{ 60.0 55.0 }
{ 60.0 60.0 }
{ 58.0 75.0 }
{ 60.0 80.0 }
{ 62.0 80.0 }
{ 65.0 82.0 }
{ 67.0 85.0 }
{ 65.0 85.

In [192]:
data_points = []
for i in range(len(points) - 1):
    p1 = (points[i][0], points[i][1])
    p2 = (points[i+1][0], points[i+1][1])    
    l = (p1, p2)
    data_points.append(l)
data_points

[((40.0, 50.0), (30.0, 50.0)),
 ((30.0, 50.0), (25.0, 50.0)),
 ((25.0, 50.0), (25.0, 52.0)),
 ((25.0, 52.0), (23.0, 52.0)),
 ((23.0, 52.0), (20.0, 50.0)),
 ((20.0, 50.0), (8.0, 45.0)),
 ((8.0, 45.0), (5.0, 45.0)),
 ((5.0, 45.0), (0.0, 45.0)),
 ((0.0, 45.0), (0.0, 40.0)),
 ((0.0, 40.0), (2.0, 40.0)),
 ((2.0, 40.0), (5.0, 35.0)),
 ((5.0, 35.0), (8.0, 40.0)),
 ((8.0, 40.0), (10.0, 40.0)),
 ((10.0, 40.0), (10.0, 35.0)),
 ((10.0, 35.0), (25.0, 35.0)),
 ((25.0, 35.0), (26.0, 32.0)),
 ((26.0, 32.0), (25.0, 30.0)),
 ((25.0, 30.0), (28.0, 30.0)),
 ((28.0, 30.0), (30.0, 30.0)),
 ((30.0, 30.0), (32.0, 30.0)),
 ((32.0, 30.0), (30.0, 32.0)),
 ((30.0, 32.0), (28.0, 35.0)),
 ((28.0, 35.0), (30.0, 35.0)),
 ((30.0, 35.0), (33.0, 35.0)),
 ((33.0, 35.0), (33.0, 32.0)),
 ((33.0, 32.0), (35.0, 32.0)),
 ((35.0, 32.0), (35.0, 30.0)),
 ((35.0, 30.0), (38.0, 15.0)),
 ((38.0, 15.0), (35.0, 5.0)),
 ((35.0, 5.0), (38.0, 5.0)),
 ((38.0, 5.0), (40.0, 5.0)),
 ((40.0, 5.0), (44.0, 5.0)),
 ((44.0, 5.0), (42.0, 10.0)),

In [193]:
'''
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7),
                  (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), 
                  (12, 13), (13, 14), (14, 15), (15, 16), (16, 17),
                  (17, 18), (18, 19), (19, 20), (20, 21), (21, 22),
                  (22, 23), (23, 24), (24, 25), (25, 26), (26, 27),
                  (27, 28), (28, 29), (29, 30), (30, 31), (31, 32),
                  (32, 33), (33, 34), (34, 35), (35, 36), (36, 37),
                  (37, 38), (38, 39), (39, 40), (40, 41), (41, 42),
                  (42, 43), (43, 44), (44, 45), (45, 46), (46, 47),
                  (47, 48), (48, 49), (49, 50), (50, 51), (51, 52),
                  (52, 53), (53, 54), (54, 55), (55, 56), (56, 57),
                  (57, 58), (58, 59), (59, 60), (60, 61), (61, 62),
                  (62, 63), (63, 64), (64, 65), (65, 66), (66, 67),
                  (67, 68), (68, 69), (69, 70), (70, 71), (71, 72),
                  (72, 73), (73, 74), (74, 75), (75, 76), (76, 77),
                  (77, 78), (78, 79), (79, 80), (80, 81), (81, 82),
                  (82, 83), (83, 84), (84, 85), (85, 86), (86, 87),
                  (87, 88), (88, 89), (89, 90), (90, 91), (91, 92),
                  (92, 93), (93, 94), (94, 95), (95, 96), (96, 97),
                  (97, 98), (98, 99), (99, 100), (100, 101), 
                  (101, 1)
                 ])

#pos = nx.spring_layout(G, k=0.1, iterations=60)

#plt.gca().invert_yaxis()
#plt.gca().invert_xaxis()

#nx.draw(G, pos, with_labels=False, font_weight='bold')
#labels = nx.draw_networkx_labels(G, pos=nx.spring_layout(G), font_size=1)
'''

"\nimport networkx as nx\nimport matplotlib.pyplot as plt\n\nG = nx.Graph()\nG.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7),\n                  (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), \n                  (12, 13), (13, 14), (14, 15), (15, 16), (16, 17),\n                  (17, 18), (18, 19), (19, 20), (20, 21), (21, 22),\n                  (22, 23), (23, 24), (24, 25), (25, 26), (26, 27),\n                  (27, 28), (28, 29), (29, 30), (30, 31), (31, 32),\n                  (32, 33), (33, 34), (34, 35), (35, 36), (36, 37),\n                  (37, 38), (38, 39), (39, 40), (40, 41), (41, 42),\n                  (42, 43), (43, 44), (44, 45), (45, 46), (46, 47),\n                  (47, 48), (48, 49), (49, 50), (50, 51), (51, 52),\n                  (52, 53), (53, 54), (54, 55), (55, 56), (56, 57),\n                  (57, 58), (58, 59), (59, 60), (60, 61), (61, 62),\n                  (62, 63), (63, 64), (64, 65), (65, 66), (66, 67),\n                  (67, 68), (68,

In [194]:
'''
true_pos = {1:(40.0, 50.0), 2:(36.0, 40.0), 3:(33.0, 35.0), 
            4:(30.0, 35.0), 5:(25.0, 35.0), 6:(26.0, 32.0),
            7:(25.0, 30.0), 8:(30.0, 30.0), 9:(33.0, 32.0),
            10:(35.0, 32.0), 11:(34.0, 25.0), 12:(32.0, 20.0),
            13:(36.0, 18.0), 14:(38.0, 15.0), 15:(38.0, 10.0),
            16:(35.0, 5.0), 17:(38.0, 5.0), 18:(40.0, 5.0),
            19:(44.0, 5.0), 20:(42.0, 10.0), 21:(42.0, 15.0),
            22:(48.0, 20.0), 23:(44.0, 25.0), 24:(42.0, 30.0),
            25:(48.0, 30.0), 26:(50.0, 30.0), 27:(53.0, 30.0),
            28:(53.0, 35.0), 29:(50.0, 35.0), 30:(47.0, 35.0),
            31:(45.0, 35.0), 32:(47.0, 40.0), 33:(50.0, 40.0),
            34:(62.0, 40.0), 35:(64.0, 46.0), 36:(72.0, 45.0),
            37:(78.0, 40.0), 38:(85.0, 35.0), 39:(87.0, 30.0),
            40:(85.0, 25.0), 41:(92.0, 30.0), 42:(95.0, 30.0),
            43:(95.0, 35.0), 44:(90.0, 35.0), 45:(86.0, 46.0),
            46:(75.0, 55.0), 47:(72.0, 55.0), 48:(70.0, 58.0),
            49:(65.0, 60.0), 50:(66.0, 55.0), 51:(60.0, 55.0),
            52:(60.0, 60.0), 53:(56.0, 64.0), 54:(60.0, 66.0),
            55:(62.0, 69.0), 56:(58.0, 70.0), 57:(58.0, 75.0),
            58:(62.0, 80.0), 59:(65.0, 82.0), 60:(67.0, 85.0),
            61:(60.0, 85.0), 62:(55.0, 85.0), 63:(55.0, 80.0),
            64:(52.0, 75.0), 65:(45.0, 70.0), 66:(45.0, 65.0),
            67:(42.0, 65.0), 68:(35.0, 66.0), 69:(35.0, 69.0),
            70:(28.0, 70.0), 71:(22.0, 75.0), 72:(20.0, 80.0),
            73:(25.0, 85.0), 74:(22.0, 85.0), 75:(20.0, 85.0),
            76:(15.0, 80.0), 77:(15.0, 75.0), 78:(18.0, 75.0),
            79:(22.0, 66.0), 80:(14.0, 66.0), 81:(8.0, 62.0),
            82:(4.0, 55.0), 83:(8.0, 45.0), 84:(5.0, 45.0),
            85:(0.0, 45.0), 86:(0.0, 40.0), 87:(2.0, 40.0),
            88:(5.0, 35.0), 89:(10.0, 35.0), 90:(8.0, 40.0),
            91:(10.0, 40.0), 92:(16.0, 42.0), 93:(20.0, 50.0),
            94:(20.0, 55.0), 95:(23.0, 52.0), 96:(25.0, 50.0),
            97:(30.0, 50.0), 98:(28.0, 52.0), 99:(30.0, 56.0), 
            100:(34.0, 60.0), 101:(42.0, 58.0)
           } 
'''

'\ntrue_pos = {1:(40.0, 50.0), 2:(36.0, 40.0), 3:(33.0, 35.0), \n            4:(30.0, 35.0), 5:(25.0, 35.0), 6:(26.0, 32.0),\n            7:(25.0, 30.0), 8:(30.0, 30.0), 9:(33.0, 32.0),\n            10:(35.0, 32.0), 11:(34.0, 25.0), 12:(32.0, 20.0),\n            13:(36.0, 18.0), 14:(38.0, 15.0), 15:(38.0, 10.0),\n            16:(35.0, 5.0), 17:(38.0, 5.0), 18:(40.0, 5.0),\n            19:(44.0, 5.0), 20:(42.0, 10.0), 21:(42.0, 15.0),\n            22:(48.0, 20.0), 23:(44.0, 25.0), 24:(42.0, 30.0),\n            25:(48.0, 30.0), 26:(50.0, 30.0), 27:(53.0, 30.0),\n            28:(53.0, 35.0), 29:(50.0, 35.0), 30:(47.0, 35.0),\n            31:(45.0, 35.0), 32:(47.0, 40.0), 33:(50.0, 40.0),\n            34:(62.0, 40.0), 35:(64.0, 46.0), 36:(72.0, 45.0),\n            37:(78.0, 40.0), 38:(85.0, 35.0), 39:(87.0, 30.0),\n            40:(85.0, 25.0), 41:(92.0, 30.0), 42:(95.0, 30.0),\n            43:(95.0, 35.0), 44:(90.0, 35.0), 45:(86.0, 46.0),\n            46:(75.0, 55.0), 47:(72.0, 55.0), 48:

In [195]:
'''
#try:
#    pos = nx.kamada_kawai_layout(G)
#except ValueError:
#    pos = nx.shell_layout(G)
#pos = nx.shell_layout(G)
plt.subplots(figsize=(20,20))

#nx.draw_networkx(G, pos=pos, with_labels=False)

#nx.draw(G, pos, with_labels=False, font_weight='bold')
nx.draw_networkx(G, pos=true_pos)
plt.show()
'''

"\n#try:\n#    pos = nx.kamada_kawai_layout(G)\n#except ValueError:\n#    pos = nx.shell_layout(G)\n#pos = nx.shell_layout(G)\nplt.subplots(figsize=(20,20))\n\n#nx.draw_networkx(G, pos=pos, with_labels=False)\n\n#nx.draw(G, pos, with_labels=False, font_weight='bold')\nnx.draw_networkx(G, pos=true_pos)\nplt.show()\n"

In [196]:
#print(pos)